In [ ]:
import sys
import os
from pathlib import Path
import pickle
import torch
import pprint
sys.path.append(os.getcwd())
sys.path.append(str(Path(os.getcwd()).parent))
from utils import *
from configuration import *
import numpy as np

if not os.path.exists('./evaluation/'):
    os.makedirs('./evaluation/')

In [ ]:
args = parser.parse_args([])
args.task = 'temporized'
args.metric = 'nominal_snn'
args.SoftEva = True
args = FormulateArgs(args)

In [ ]:
results = torch.zeros([13,10,4])

for seed in range(10):
    for ds in range(13):
        args.SEED = seed
        args.DATASET = ds

        valid_loader, datainfo = GetDataLoader(args, 'valid', path='../dataset')
        test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset')

        modelname = f"model_SNN_data_{datainfo['dataname']}_seed_{args.SEED:02d}.model"
        
        
        if os.path.isfile(f'./models/{modelname}'):                
            print(modelname)
            
            model = torch.load(f'./models/{modelname}', map_location=args.DEVICE)
            model.UpdateArgs(args)

            SetSeed(args.SEED)

            evaluator = Evaluator(args).to(args.DEVICE)

            for x,y in valid_loader:
                X_valid, y_valid = x.to(args.DEVICE), y.to(args.DEVICE)
            for x,y in test_loader:
                X_test, y_test = x.to(args.DEVICE), y.to(args.DEVICE)

            acc_valid, power_valid = evaluator(model, X_valid, y_valid)
            acc_test, power_test   = evaluator(model, X_test,  y_test)

            results[ds,seed,0] = acc_valid
            results[ds,seed,1] = acc_test
            results[ds,seed,2] = power_valid
            results[ds,seed,3] = power_test
            
        else:
            results[ds,seed,:] = float('nan')

In [ ]:
results.shape

In [ ]:
valid_acc_data = results[:, :, 0]
best_seeds = torch.argmax(valid_acc_data, dim=1)
best_seeds

In [ ]:
best_data = torch.zeros(13, 4)
for i, seed in enumerate(best_seeds):
    best_data[i] = results[i, seed]
best_data_advanced = results[torch.arange(13), best_seeds]
best_data_advanced

In [ ]:
best_data_advanced.mean(0)

In [ ]:
results_std = torch.std(results, dim=1)

In [ ]:
results_std

In [ ]:
final = torch.cat([best_data_advanced, results_std], dim=1)
final.shape

In [ ]:
np.savetxt('./result.txt', final.numpy(), fmt='%.9f', delimiter='\t')